In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from PIL import Image
import numpy as np
from torchvision.io import read_image
from matplotlib import image
from prepare_dataset import MyDataset,subset_generator
from utils import display_some_images,stats_of_image_and_label,MyCuda_Stats,get_output_features_of_model
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from Model import Network,new_model
from train_fn import train,validation,image_validation,validation_classes,train_batch
from tqdm import tqdm


In [2]:
fullvgg=full_vgg()
total_trainable_parameters(fullvgg)
total_trainable_parameters(fullvgg.features)
total_trainable_parameters(vgg_all_freezed())
total_trainable_parameters(vgg_layer_freeze(24))


Total trainable parameters of VGG is 	 138357544

Total trainable parameters of Sequential is 	 14714688

Total trainable parameters of Sequential is 	 0

Total trainable parameters of Sequential is 	 7079424


In [3]:
# DATA LOADER PARAMETERS
batch=16
CUDA_LAUNCH_BLOCKING=1 

In [4]:
MyCuda_Stats()

No of GPUs i have is 1
0
My Graphic Card is NVIDIA GeForce GTX 1050 Ti
Is Cuda Available True


In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])

In [6]:
anno_dir=r"F:\Affectnet\annotations"
image_dir=r"F:\Affectnet\train_set\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)

In [7]:
val_anno_dir=r"F:\Affectnet\val_set\val_set\annotations"
val_image_dir=r"F:\Affectnet\val_set\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

In [8]:
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [9]:
print(len(train_loader))
print(len(val_loader))


17979
250


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [11]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)



Total trainable parameters of Sequential is 	 12979200


In [12]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [13]:
model1=new_model(feature_extractor,input_features_for_denselayer).to(device)


In [14]:
total_trainable_parameters(model1)


Total trainable parameters of new_model is 	 115776520


In [15]:
weights=[0.1,0.1,0.5,0.5,0.8,0.99,0.5,0.99]
loss1=nn.CrossEntropyLoss(weight=torch.tensor(weights).to(device))
loss_plain=nn.CrossEntropyLoss()
optimizer=optim.SGD(model1.parameters(),lr=0.0001,momentum=0.9)
classes=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [16]:
#model1.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\modelrecent.pth"))

In [17]:
#optimizer.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizer1.pth"))

In [ ]:
# NO OF EPOCHS IS 5
epochs=3
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,model1,loss1,optimizer)
    validation(val_loader,model1,loss1)
    
    


Epoch-0


  0%|                                                                            | 1/17979 [00:23<118:13:42, 23.67s/it]

loss:2.177055 batch:0/17979


  1%|▋                                                                           | 161/17979 [02:33<4:01:14,  1.23it/s]

loss:1.798539 batch:160/17979


...

 12%|█████████                                                                | 2241/17979 [2:50:02<3:40:25,  1.19it/s]

loss:1.480303 batch:2240/17979


 13%|█████████▋                                                               | 2401/17979 [2:54:57<7:00:13,  1.62s/it]

loss:1.709017 batch:2400/17979


 14%|██████████▎                                                              | 2550/17979 [2:59:47<4:25:35,  1.03s/it]

In [ ]:
validation_classes(val_class_loader,model1,loss1,0)

In [ ]:
validation_classes(val_class_loader,model1,loss1,1)

In [ ]:
validation_classes(val_class_loader,model1,loss1,2)

In [ ]:
validation_classes(val_class_loader,model1,loss1,3)

In [ ]:
validation_classes(val_class_loader,model1,loss1,4)

In [ ]:
validation_classes(val_class_loader,model1,loss1,5)

In [ ]:
validation_classes(val_class_loader,model1,loss1,6)

In [ ]:
validation_classes(val_class_loader,model1,loss1,7)

In [ ]:
torch.save(model1.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\model(weightedloss1adam.pth")

In [ ]:


torch.save(optimizer.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizer(weightedloss1adam)..pth")